In [1]:
import os
import numpy as np
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
#sns.set(style="darkgrid")
from keras.models import load_model



Using TensorFlow backend.


In [17]:
df = pd.read_excel("time_series.xlsx")

In [18]:
df = df.iloc[:,1:2881]

In [4]:
df = df.dropna()

In [19]:
df[:60]

,0,1,2,3,4,5,6,7,8,9,...,2870,2871,2872,2873,2874,2875,2876,2877,2878,2879
0,34.398087,34.398087,34.398087,34.398087,37.958370,37.958370,37.958370,37.958370,35.538284,35.538284,...,49.292221,49.292221,49.292221,43.439564,43.439564,43.439564,32.491669,32.491669,32.491669,32.491669
1,11.923971,11.923971,11.923971,11.923971,11.109582,11.109582,11.109582,11.109582,16.095539,16.095539,...,25.328291,25.328291,25.328291,30.823246,30.823246,30.823246,39.288010,39.288010,39.288010,39.288010
2,58.222057,58.222057,58.222057,58.222057,53.618980,53.618980,53.618980,53.618980,51.198895,51.198895,...,122.791733,122.791733,122.791733,125.047371,125.340759,125.340759,125.290382,125.290382,125.290382,125.290382
3,15.012849,15.012849,15.012849,15.012849,11.843982,11.843982,11.843982,11.843982,4.357113,4.357113,...,22.273878,22.273878,22.273878,22.045980,22.045980,22.045980,16.569902,16.569902,16.569902,16.569902
4,28.089104,28.089104,28.089104,28.089104,27.041897,27.041897,27.041897,27.041897,23.007502,23.007502,...,49.777802,49.777802,49.777802,51.152920,51.152920,51.152920,48.955788,48.955788,48.955788,48.955788
5,82.762115,82.762115,82.762115,82.762115,80.149406,80.149406,80.149406,80.149406,76.459694,76.459694,...,161.085953,161.085953,161.085953,189.291138,189.291138,189.291138,216.766022,216.766022,216.766022,216.766022
6,38.344837,38.344837,38.344837,38.344837,38.344837,0.038345,38.344837,38.344837,38.344837,38.344837,...,66.964516,66.964516,66.964516,64.905479,64.905479,64.905479,70.762062,70.762062,70.762062,70.762062
7,28.514704,28.514704,28.514704,28.514704,26.836140,26.836140,26.836140,26.836140,26.110458,26.110458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,46.563629,46.563629,46.563629,46.563629,48.463173,48.463173,48.463173,48.463173,45.526272,45.526272,...,82.821243,82.821243,82.821243,91.866982,91.866982,91.866982,99.880157,99.880157,99.880157,99.880157
9,7.824219,7.824219,7.824219,7.824219,7.824219,7.824219,7.824219,7.824219,7.824219,7.824219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
def isSubArray(A, B, n, m):
    ii = 0
    j = 0
    while (ii < n and j < m): 
        if A[ii] == B[j]: 
            ii += 1 
            j += 1 
            if (j == m): 
                return True 
        else: 
            ii -= j-1 
            j = 0 
    return False 

In [26]:
for i in df.index:
    xx = df.iloc[i][:250]
    #if any(np.equal(xx,[0,0,0,0]).all(1)):
    #vv = np.isin(xx,[0,0,0,0])
    #if [0.0,0.0,0.0,0.0] in xx.tolist():
    #print(xx)
    #array123 = lambda x: any([0,0,0,0] == x[offset:offset+3] for offset in range(len(x)))
    print(i, isSubArray(xx.tolist(),[0,0,0,0], 250, 4))

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 True
12 False
13 True
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 True
22 False
23 False
24 False
25 False
26 False
27 False
28 True
29 False
30 True
31 False
32 False
33 False
34 True
35 False
36 False
37 False
38 False
39 False
40 False
41 True
42 False
43 False
44 False
45 True
46 False
47 True
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 True
59 False
60 False
61 False
62 False
63 False
64 False
65 False
66 False
67 True
68 False
69 True
70 False
71 True
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 False
81 False
82 False
83 False
84 False
85 True
86 False
87 False
88 False
89 False
90 False
91 True
92 True
93 False
94 False
95 False
96 False
97 True
98 False
99 False
100 False
101 True
102 False
103 False
104 True
105 True
106 True
107 False
108 False
109 False
110 True
111 True
112 True
113 

In [22]:
n = 7
m = 4 
A = [2, 3, 0, 5, 1, 1, 2] 
B = [3, 0, 5, 1] 
if (isSubArray(A, B, n, m)): 
    print("YES") 
else:
    print("NO") 

YES
